In [1]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
sid = SentimentIntensityAnalyzer()

In [3]:
df = pd.read_json('amelia_evandro.json')
df.replace(r'\n', '', regex=True, inplace=True)
df.replace(r'\xa0', '', regex=True, inplace=True)
df.replace(r'\r', '', regex=True, inplace=True)
df.head()

,Title,Date,Contributor,Content,Link,Role
0,FPT Software Partners With Amelia to Enhance C...,"October 20, 2021 • 2 minute read",,Partnership will expand FPT Software’s client ...,https://amelia.ai/press-release/fpt-software-p...,
1,Amelia Hired By Staffmark Group to Bring Conve...,"July 29, 2021 • 2 minute read",,Staffmark Group engages Amelia to deliver inte...,https://amelia.ai/press-release/amelia-hired-b...,
2,OrangeShine Hires Amelia to Automate IT Operat...,"May 28, 2021 • 2 minute read",,The leading B2B online fashion wholesale marke...,https://amelia.ai/press-release/orangeshine-hi...,
3,Amelia Recognized as a Leader in Intelligent V...,"May 10, 2021 • 2 minute read",,New report on Hyper-Intelligent Automation hig...,https://amelia.ai/press-release/amelia-recogni...,
4,Kenneth S. Nugent Law Firm Forms Subcompany 1L...,"April 26, 2021 • 2 minute read",,1LAW provides Amelia as a platform that law fi...,https://amelia.ai/press-release/kenneth-s-nuge...,


In [4]:
df.isnull().sum()

Title          0
Date           0
Contributor    0
Content        0
Link           0
Role           0
dtype: int64

In [5]:
df.dropna(inplace=True)
df.drop(['Role', 'Contributor'], axis=1, inplace=True)

In [6]:
sid.polarity_scores(df.iloc[0]['Content'])

{'neg': 0.0, 'neu': 0.869, 'pos': 0.131, 'compound': 0.9955}

In [7]:
df.Content[0]

'Partnership will expand FPT Software’s client offerings with Amelia’s market-leading AI solutions.NEW YORK, NY, October 20, 2021 — Amelia, a leading Enterprise AI software company, today announced its new partnership with FPT Software, Vietnam’s largest technology services provider. FPT Software provides technical consulting, streamlining and cost-savings services to thousands of global customers, including Fortune Global 500 companies.FPT Software will include Amelia’s industry-leading AI solutions within its client offerings. Through this partnership, the provider will enhance its services by providing clients with the most modern, capable and transformative solutions. By leveraging its “people, process and technology” approach and Amelia’s solutions, FPT Software will help its customers streamline operations and reduce costs.“We are excited to partner withAmeliaand incorporate their Conversational AI capabilities into our intelligent automation solutions along with our akaBotTM pla

In [8]:
df['Scores'] = df['Content'].apply(lambda content: sid.polarity_scores(content))
df['Date']= df['Date'].str.strip('• 1|2|3|4|5 minute read')
df['Date']= pd.to_datetime(df['Date'])

In [9]:
df.head()

,Title,Date,Content,Link,Scores
0,FPT Software Partners With Amelia to Enhance C...,2020-10-20,Partnership will expand FPT Software’s client ...,https://amelia.ai/press-release/fpt-software-p...,"{'neg': 0.0, 'neu': 0.869, 'pos': 0.131, 'comp..."
1,Amelia Hired By Staffmark Group to Bring Conve...,2020-07-29,Staffmark Group engages Amelia to deliver inte...,https://amelia.ai/press-release/amelia-hired-b...,"{'neg': 0.004, 'neu': 0.87, 'pos': 0.126, 'com..."
2,OrangeShine Hires Amelia to Automate IT Operat...,2020-05-28,The leading B2B online fashion wholesale marke...,https://amelia.ai/press-release/orangeshine-hi...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp..."
3,Amelia Recognized as a Leader in Intelligent V...,2020-05-10,New report on Hyper-Intelligent Automation hig...,https://amelia.ai/press-release/amelia-recogni...,"{'neg': 0.003, 'neu': 0.898, 'pos': 0.099, 'co..."
4,Kenneth S. Nugent Law Firm Forms Subcompany 1L...,2020-04-26,1LAW provides Amelia as a platform that law fi...,https://amelia.ai/press-release/kenneth-s-nuge...,"{'neg': 0.015, 'neu': 0.857, 'pos': 0.128, 'co..."


In [10]:
df['Compound'] = df['Scores'].apply(lambda d: d['compound'])
df['Overall Score'] = df['Compound'].apply(lambda score: 'POS' if score >=0 else 'NEG')

In [11]:
df[df['Overall Score'] == 'NEG']

,Title,Date,Content,Link,Scores,Compound,Overall Score


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=.9, min_df=.1, stop_words= 'english')
dtm = cv.fit_transform(df['Content'])

In [13]:
dtm

<197x379 sparse matrix of type '<class 'numpy.int64'>'
	with 20017 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=10)
LDA.fit(dtm)

LatentDirichletAllocation()

In [15]:
first_topic = LDA.components_[0]
first_topic.argsort()

array([266, 374,  36, 252, 120, 334, 271, 352, 314, 161, 207,  88,  58,
        25,  78, 289, 238, 233,  46,  90, 303,  74, 183, 299, 179, 173,
       199,  92, 251, 343, 197, 363,  28,  93,  32, 288,   0, 107, 240,
       132, 141, 241, 309, 279, 245,  67, 336, 105, 232,  17, 124, 348,
       302,  66, 239,  54, 300, 200, 344, 208, 350,  39, 357, 312,  10,
       319,  16, 375,  35, 270, 133, 169, 364, 280, 325, 112, 257, 246,
        12, 187, 201,  14, 291, 153, 304,  42, 152, 356, 370, 365,  23,
       108, 269, 247, 293,  30,  22, 212, 317, 341,  45,  98, 273, 367,
       354,  34,  76, 134,  84,  68, 286, 106,  20, 229,  60, 170, 175,
       215, 102, 225, 135,  85, 202, 186, 163, 131, 355, 321,   9,  61,
       176, 359,  62, 158, 109, 181, 206, 114,  99, 167,  40,  83,  65,
       281, 310, 100, 125, 323, 243,   3, 110, 191, 276, 168, 209,  77,
       166, 295, 111, 140, 349, 263,   8, 335,  38,  96,  21, 121, 171,
       351,  15, 262,  19, 210, 268, 164, 372, 261, 144, 104, 22

In [16]:
top_ten = first_topic.argsort()[-10:]
for index in top_ten:
    print(cv.get_feature_names_out()[index])

digital
enterprise
automation
end
leading
operations
technology
new
ipsoft
amelia


In [17]:
import sys

orig_stdout = sys.stdout
f = open(
    'df.txt', 'w'
)
sys.stdout = f


for index, topic in enumerate(LDA.components_):
    dataframe = pd.DataFrame(columns=[f'THE TOP 15 WORDS FOR TOPIC #{index}'], data=[cv.get_feature_names_out()[index] for index in topic.argsort()[-15:]])
    print(dataframe)

sys.stdout = orig_stdout
f.close()

In [18]:
df2 = pd.read_table('df.txt', header=None)
import numpy as np
arr = np.array(df2)
dataframe = pd.DataFrame(arr.reshape((10, 16))).T
headers = dataframe.iloc[0]
new_df = pd.DataFrame(data=dataframe.values[1:], columns=headers)
new_df

,THE TOP 15 WORDS FOR TOPIC #0,THE TOP 15 WORDS FOR TOPIC #1,THE TOP 15 WORDS FOR TOPIC #2,THE TOP 15 WORDS FOR TOPIC #3,THE TOP 15 WORDS FOR TOPIC #4,THE TOP 15 WORDS FOR TOPIC #5,THE TOP 15 WORDS FOR TOPIC #6,THE TOP 15 WORDS FOR TOPIC #7,THE TOP 15 WORDS FOR TOPIC #8,THE TOP 15 WORDS FOR TOPIC #9
0,0 innovation,0 productivity,0 leading,0 recent,0 time,0 available,0 customers,0 language,0 ed,0 computing
1,1 global,1 issues,1 intelligent,1 like,1 information,1 user,1 company,1 virtual,1 op,1 sure
2,2 world,2 technologies,2 ipsoft,2 value,2 new,2 processes,2 experiences,2 support,2 virtual,2 read
3,3 solutions,3 new,3 human,3 employees,3 recent,3 new,3 operations,3 natural,3 intelligent,3 recent
4,4 cognitive,4 future,4 end,4 work,4 op,4 able,4 enterprise,4 service,4 service,4 machine
5,5 digital,5 roles,5 leader,5 intelligence,5 ed,5 chatbots,5 services,5 ipsoft,5 businesses,5 cognitive
6,6 enterprise,6 tasks,6 everest,6 information,6 services,6 ipsoft,6 new,6 insurance,6 read,6 director
7,7 automation,7 ipsoft,7 enterprise,7 technology,7 managing,7 questions,7 human,7 companies,7 companies,7 intelligence
8,8 end,8 workers,8 group,8 business,8 ipsoft,8 context,8 digital,8 agents,8 voice,8 conversational
9,9 leading,9 employees,9 market,9 human,9 impact,9 author,9 conversational,9 cognitive,9 business,9 says


In [19]:
topic_results = LDA.transform(dtm)
topic_results[0].argmax()

6

In [20]:
df['Topic'] = topic_results.argmax(axis=1)

In [23]:
positive = df[df['Overall Score'] == 'POS']
positive.reset_index(drop=True, inplace=True)
negative = df[df['Overall Score'] == 'NEG']
negative.reset_index(drop=True, inplace=True)
analytics = df.groupby('Date').count().sort_values(by='Title', ascending=False)
analytics.to_excel('amelia_alytics.xlsx')
df.to_excel('amelia_final.xlsx', index=False)
positive.to_excel('amelia_positive_articles.xlsx', index=False)
negative.to_excel('amelia_negative_articles.xlsx', index=False)
new_df.to_excel('amelia_topic.xlsx', index=False)